In [1]:
import numpy as np
import cv2 as cv
from skimage.feature import canny

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

import joblib

## **Ekstrasi Fitur**

In [2]:
def face_recognition(image_path):
    # Memuat Haar Cascade untuk deteksi wajah manusia atau kucing (Jika ada)
    # cascade_wajah = cv.CascadeClassifier('/content/drive/MyDrive/PCVK_2024/Images/haarcascades/haarcascade_frontalface_default.xml')
    cascade_wajah = cv.CascadeClassifier('input_output/haarcascade_frontalface_default.xml')

    # Membaca Gambar
    image = cv.imread(image_path)

    # Mengubah gambar menjadi grayscale
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # scaleFactor: Faktor skala untuk memperkecil gambar setiap proses deteksi.
    # minNeighbors: Jumlah minimum tetangga yang diperlukan untuk validasi objek.
    # minSize: Ukuran minimum objek yang akan dideteksi.
    faces = cascade_wajah.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Wajah tidak terdeteksi
    if len(faces) == 0:
        print("Wajah tidak terdeteksi!")
        return None

    # Memproses setiap wajah yang terdeteksi
    for (x, y, w, h) in faces:
        # Memotong gambar sesuai area deteksi wajah
        face_crop = image[y:y+h, x:x+w]
        face_crop = cv.resize(face_crop, (200,200))

        # Mengurangi area gambar sebesar 10% (90% dari ukuran asli)
        height, width = face_crop.shape[:2]
        new_height = int(height * 0.70)
        new_width = int(width * 0.70)

        # Menghitung margin untuk cropping agar tetap di tengah
        top_margin = (height - new_height) // 2
        left_margin = (width - new_width) // 2

        # Memotong area gambar
        face_crop = face_crop[top_margin:top_margin + new_height, left_margin:left_margin + new_width]

    return face_crop

In [3]:
import random

def augment_image(image):
    augmented_image = image.copy()

    # Random horizontal flip
    if random.random() > 0.5:
        augmented_image = cv.flip(augmented_image, 1)

    # Random brightness/contrast adjustment
    if random.random() > 0.5:
        alpha = random.uniform(0.7, 1.3)  # Random contrast factor
        beta = random.randint(-50, 50)    # Random brightness factor
        augmented_image = cv.convertScaleAbs(augmented_image, alpha=alpha, beta=beta)

    # Random rotation (±15 degrees)
    if random.random() > 0.5:
        angle = random.randint(-15, 15)  # Random angle for rotation
        h, w = augmented_image.shape[:2]
        center = (w // 2, h // 2)
        rotation_matrix = cv.getRotationMatrix2D(center, angle, 1.0)
        augmented_image = cv.warpAffine(augmented_image, rotation_matrix, (w, h))

    # Random Gaussian blur
    if random.random() > 0.5:
        ksize = random.choice([3, 5, 7])  # Random kernel size
        augmented_image = cv.GaussianBlur(augmented_image, (ksize, ksize), 0)

    return augmented_image

In [4]:
def features_grid(img):
    features = np.array([], dtype='uint8')

    for y in range(0, img.shape[0], 10):
        for x in range(0, img.shape[1], 10):

            # Cropping the image into a section.
            section_img = img[y:y+10, x:x+10]

            # Claculating the mean and stdev of the sectioned image.
            section_mean = np.mean(section_img)
            section_std = np.std(section_img)

            # Appending the above calculated values into features array.
            features = np.append(features, [section_mean, section_std])

    # Returning the features array.
    return features

In [5]:
def extract_canny_edges(image_path, augment=False):
    img = face_recognition(image_path)
    if img is None:
        return None

    if augment:
        img = augment_image(img)

    # Mengonversi gambar ke grayscale
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Meresize gambar ke ukuran 200x200 piksel
    img = cv.resize(img, (200, 200))

    # Mengonversi gambar ke gambar dengan filter canny
    img = canny(img, sigma=0.9)

    # Ekstraksi fitur (mean dan stdev dari grid 10x10 piksel)
    img_features = features_grid(img)

    return img_features

In [6]:
def classify_image(image_path, model, augment=False):
    """
    Fungsi untuk mengklasifikasikan gambar menggunakan model yang telah dilatih.
    """
    features = extract_canny_edges(image_path, augment=augment)

    if features is None:
        return "Wajah tidak terdeteksi!"

    features = features.reshape(1, -1)
    label = model.predict(features)[0]
    return label

In [7]:
import matplotlib.pyplot as plt

# Fungsi untuk menampilkan gambar dengan ukuran lebih kecil
def display_image(image, title="Image", figsize=(10, 10)):
    plt.figure(figsize=figsize)
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

# Akurasi

## New (Sudah + Canny)

### XGBOOST

In [9]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score

# Fungsi untuk membaca dataset lokal dan menyiapkan filepath serta label
def load_dataset(folder_path):
    data = []
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(root, file_name)
                # Ekstrak label dari angka sebelum ekstensi
                label = int(file_name.split('_')[-1].split('.')[0])
                data.append((file_path, label))  # Path dan label
    return pd.DataFrame(data, columns=['filepath', 'label'])

# Fungsi untuk memproses dataset dan menghitung akurasi
def evaluate_model(folder_path, model):
    data = load_dataset(folder_path)
    y_true = []
    y_pred = []

    for _, row in data.iterrows():
        image_path = row['filepath']
        true_label = row['label']

        # Prediksi dengan augmentasi dan tanpa augmentasi
        predicted_label_original = classify_image(image_path, model, augment=False)
        # predicted_label_augmented = classify_image(image_path, model, augment=True)

        # Menambahkan hasil prediksi ke daftar
        y_true.append(true_label)
        y_pred.append(predicted_label_original)

    # Hitung akurasi
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy, y_true, y_pred

# Path dataset lokal
train_folder = 'dataset/final_dataset/train'
test_folder = 'dataset/final_dataset/test'

# Memuat model
model_path = "model/xgboost_canny_model_acc_0.5.pkl"
model_xgb = joblib.load(model_path)

# Evaluasi model dengan dataset test
accuracy, y_true, y_pred = evaluate_model(test_folder, model_xgb)

# Tampilkan hasil
print(f"Akurasi Model: {accuracy * 100:.2f}%")

Akurasi Model: 16.57%


### SVM

In [40]:
# Memuat model
model_path = "model/svm_canny_model_acc_0.56.pkl"
model_svm = joblib.load(model_path)

# Evaluasi model dengan dataset test
accuracy, y_true, y_pred = evaluate_model(test_folder, model_svm)

# Tampilkan hasil
print(f"Akurasi Model: {accuracy * 100:.2f}%")

Akurasi Model: 17.12%


### RF

In [41]:
# Memuat model
model_path = "model/rf_canny_model_acc_0.56.pkl"
model_rf = joblib.load(model_path)

# Evaluasi model dengan dataset test
accuracy, y_true, y_pred = evaluate_model(test_folder, model_rf)

# Tampilkan hasil
print(f"Akurasi Model: {accuracy * 100:.2f}%")

Akurasi Model: 17.12%


## Tanpa Augmentasi

### XGBOOST

In [ ]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score

# Fungsi untuk membaca dataset lokal dan menyiapkan filepath serta label
def load_dataset(folder_path):
    data = []
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(root, file_name)
                # Ekstrak label dari angka sebelum ekstensi
                label = int(file_name.split('_')[-1].split('.')[0])
                data.append((file_path, label))  # Path dan label
    return pd.DataFrame(data, columns=['filepath', 'label'])

# Fungsi untuk memproses dataset dan menghitung akurasi
def evaluate_model(folder_path, model):
    data = load_dataset(folder_path)
    y_true = []
    y_pred = []

    for _, row in data.iterrows():
        image_path = row['filepath']
        true_label = row['label']

        # Prediksi dengan augmentasi dan tanpa augmentasi
        predicted_label_original = classify_image(image_path, model, augment=False)
        # predicted_label_augmented = classify_image(image_path, model, augment=True)

        # Menambahkan hasil prediksi ke daftar
        y_true.append(true_label)
        y_pred.append(predicted_label_original)

    # Hitung akurasi
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy, y_true, y_pred

# Path dataset lokal
train_folder = 'dataset/final_dataset/train'
test_folder = 'dataset/final_dataset/test'

# Memuat model
model_path = "model/xgboost_canny_model_acc.pkl"
model_xgb = joblib.load(model_path)

# Evaluasi model dengan dataset test
accuracy, y_true, y_pred = evaluate_model(test_folder, model_xgb)

# Tampilkan hasil
print(f"Akurasi Model: {accuracy * 100:.2f}%")

Akurasi Model: 16.67%


### SVM

In [22]:
# Memuat model
model_path = "model/svm_canny_model_acc.pkl"
model_svm = joblib.load(model_path)

# Evaluasi model dengan dataset test
accuracy, y_true, y_pred = evaluate_model(test_folder, model_svm)

# Tampilkan hasil
print(f"Akurasi Model: {accuracy * 100:.2f}%")

Akurasi Model: 18.50%


### RF

In [23]:
# Memuat model
model_path = "model/rf_canny_model_acc.pkl"
model_rf = joblib.load(model_path)

# Evaluasi model dengan dataset test
accuracy, y_true, y_pred = evaluate_model(test_folder, model_rf)

# Tampilkan hasil
print(f"Akurasi Model: {accuracy * 100:.2f}%")

Akurasi Model: 18.50%


## Dengan Augmentasi

### XGBOOST

In [24]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score

# Fungsi untuk membaca dataset lokal dan menyiapkan filepath serta label
def load_dataset(folder_path):
    data = []
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(root, file_name)
                # Ekstrak label dari angka sebelum ekstensi
                label = int(file_name.split('_')[-1].split('.')[0])
                data.append((file_path, label))  # Path dan label
    return pd.DataFrame(data, columns=['filepath', 'label'])

# Fungsi untuk memproses dataset dan menghitung akurasi
def evaluate_model(folder_path, model):
    data = load_dataset(folder_path)
    y_true = []
    y_pred = []

    for _, row in data.iterrows():
        image_path = row['filepath']
        true_label = row['label']

        # Prediksi dengan augmentasi dan tanpa augmentasi
        # predicted_label_original = classify_image(image_path, model, augment=False)
        predicted_label_augmented = classify_image(image_path, model, augment=True)

        # Menambahkan hasil prediksi ke daftar
        y_true.append(true_label)
        y_pred.append(predicted_label_augmented)

    # Hitung akurasi
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy, y_true, y_pred

# Path dataset lokal
train_folder = 'dataset/final_dataset/train'
test_folder = 'dataset/final_dataset/test'

# Memuat model
model_path = "model/xgboost_canny_model_acc.pkl"
model_xgb = joblib.load(model_path)

# Evaluasi model dengan dataset test
accuracy, y_true, y_pred = evaluate_model(test_folder, model_xgb)

# Tampilkan hasil
print(f"Akurasi Model: {accuracy * 100:.2f}%")

Akurasi Model: 16.67%


### SVM

In [25]:
# Memuat model
model_path = "model/svm_canny_model_acc.pkl"
model_svm = joblib.load(model_path)

# Evaluasi model dengan dataset test
accuracy, y_true, y_pred = evaluate_model(test_folder, model_svm)

# Tampilkan hasil
print(f"Akurasi Model: {accuracy * 100:.2f}%")

Akurasi Model: 17.06%


### RF

In [26]:
# Memuat model
model_path = "model/rf_canny_model_acc.pkl"
model_rf = joblib.load(model_path)

# Evaluasi model dengan dataset test
accuracy, y_true, y_pred = evaluate_model(test_folder, model_rf)

# Tampilkan hasil
print(f"Akurasi Model: {accuracy * 100:.2f}%")

Akurasi Model: 17.71%


## **Klasifikasikan**

### XGBOOST FINAL

In [ ]:
# Load model
model_path = "model/xgboost_canny_model_acc.pkl"
model_xgb = joblib.load(model_path)

print("Age Range 1")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/1/{i}.jpg'  # Update this path as needed
    predicted_label_augmented = classify_image(image_path, model_xgb, augment=True)
    print(f"image {i}")
    print(f"Prediksi Label (Augmented): {predicted_label_augmented}")
    print(f"Prediksi Label (Original): {predicted_label_original}")
    print()

Age Range 1
image 20
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 21
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 22
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 23
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 24
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 25
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 26
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 27
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 28
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1



In [ ]:
print("Age Range 2")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/2/{i}.jpg'  # Update this path as needed
    predicted_label_augmented = classify_image(image_path, model_xgb, augment=True)
    print(f"image {i}")
    print(f"Prediksi Label (Augmented): {predicted_label_augmented}")
    print(f"Prediksi Label (Original): {predicted_label_original}")
    print()

Age Range 2
image 20
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 21
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 22
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 23
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 24
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 25
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 26
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 27
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 28
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1



In [ ]:
print("Age Range 3")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/3/{i}.jpg'  # Update this path as needed
    predicted_label_augmented = classify_image(image_path, model_xgb, augment=True)
    print(f"image {i}")
    print(f"Prediksi Label (Augmented): {predicted_label_augmented}")
    print(f"Prediksi Label (Original): {predicted_label_original}")
    print()

Age Range 3
image 20
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 21
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 22
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 23
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 24
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 25
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 26
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 27
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 28
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1



In [ ]:
print("Age Range 4")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/4/{i}.jpg'  # Update this path as needed
    predicted_label_augmented = classify_image(image_path, model_xgb, augment=True)
    print(f"image {i}")
    print(f"Prediksi Label (Augmented): {predicted_label_augmented}")
    print(f"Prediksi Label (Original): {predicted_label_original}")
    print()

Age Range 4
image 20
Prediksi Label (Augmented): 0
Prediksi Label (Original): 1

image 21
Prediksi Label (Augmented): 2
Prediksi Label (Original): 1

image 22
Prediksi Label (Augmented): 2
Prediksi Label (Original): 1

image 23
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 24
Prediksi Label (Augmented): 3
Prediksi Label (Original): 1

image 25
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 26
Prediksi Label (Augmented): 0
Prediksi Label (Original): 1

image 27
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 28
Prediksi Label (Augmented): 0
Prediksi Label (Original): 1



In [ ]:
print("Age Range 5")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/5/{i}.jpg'  # Update this path as needed
    predicted_label_augmented = classify_image(image_path, model_xgb, augment=True)
    print(f"image {i}")
    print(f"Prediksi Label (Augmented): {predicted_label_augmented}")
    print(f"Prediksi Label (Original): {predicted_label_original}")
    print()

Age Range 5
image 20
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 21
Prediksi Label (Augmented): 0
Prediksi Label (Original): 1

image 22
Prediksi Label (Augmented): 0
Prediksi Label (Original): 1

image 23
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 24
Prediksi Label (Augmented): 0
Prediksi Label (Original): 1

image 25
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 26
Prediksi Label (Augmented): 1
Prediksi Label (Original): 1

image 27
Prediksi Label (Augmented): 3
Prediksi Label (Original): 1

image 28
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1



### SVM FINAL

In [ ]:
# Load model SVM
model_path = "model/svm_canny_model_acc.pkl"
model_svm = joblib.load(model_path)

print("Age Range 1")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/1/{i}.jpg'  # Update this path as needed
    predicted_label_augmented = classify_image(image_path, model_svm, augment=True)
    print(f"image {i}")
    print(f"Prediksi Label (Augmented): {predicted_label_augmented}")
    print(f"Prediksi Label (Original): {predicted_label_original}")
    print()

Age Range 1
image 20
Prediksi Label (Augmented): 1
Prediksi Label (Original): 1

image 21
Prediksi Label (Augmented): 1
Prediksi Label (Original): 1

image 22
Prediksi Label (Augmented): 1
Prediksi Label (Original): 1

image 23
Prediksi Label (Augmented): 3
Prediksi Label (Original): 1

image 24
Prediksi Label (Augmented): 3
Prediksi Label (Original): 1

image 25
Prediksi Label (Augmented): 3
Prediksi Label (Original): 1

image 26
Prediksi Label (Augmented): 3
Prediksi Label (Original): 1

image 27
Prediksi Label (Augmented): 2
Prediksi Label (Original): 1

image 28
Prediksi Label (Augmented): 3
Prediksi Label (Original): 1



In [ ]:
print("Age Range 2")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/2/{i}.jpg'  # Update this path as needed
    predicted_label_augmented = classify_image(image_path, model_svm, augment=True)
    print(f"image {i}")
    print(f"Prediksi Label (Augmented): {predicted_label_augmented}")
    print(f"Prediksi Label (Original): {predicted_label_original}")
    print()

Age Range 2
image 20
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 21
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 22
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 23
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 24
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 25
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 26
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 27
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 28
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1



In [ ]:
print("Age Range 3")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/3/{i}.jpg'  # Update this path as needed
    predicted_label_augmented = classify_image(image_path, model_svm, augment=True)
    print(f"image {i}")
    print(f"Prediksi Label (Augmented): {predicted_label_augmented}")
    print(f"Prediksi Label (Original): {predicted_label_original}")
    print()

Age Range 3
image 20
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 21
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 22
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 23
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 24
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 25
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 26
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 27
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 28
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1



In [ ]:
print("Age Range 4")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/4/{i}.jpg'  # Update this path as needed
    predicted_label_augmented = classify_image(image_path, model_svm, augment=True)
    print(f"image {i}")
    print(f"Prediksi Label (Augmented): {predicted_label_augmented}")
    print(f"Prediksi Label (Original): {predicted_label_original}")
    print()

Age Range 4
image 20
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 21
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 22
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 23
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 24
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 25
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 26
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 27
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 28
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1



In [ ]:
print("Age Range 5")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/5/{i}.jpg'  # Update this path as needed
    predicted_label_augmented = classify_image(image_path, model_svm, augment=True)
    print(f"image {i}")
    print(f"Prediksi Label (Augmented): {predicted_label_augmented}")
    print(f"Prediksi Label (Original): {predicted_label_original}")
    print()

Age Range 5
image 20
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 21
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 22
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 23
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 24
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 25
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 26
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 27
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 28
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1



### RANDOM FOREST FINAL

In [ ]:
# Load model 
model_path = "model/rf_canny_model_acc.pkl"
model_rf = joblib.load(model_path)

print("Age Range 1")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/1/{i}.jpg'  # Update this path as needed
    predicted_label_augmented = classify_image(image_path, model_rf, augment=True)
    print(f"image {i}")
    print(f"Prediksi Label (Augmented): {predicted_label_augmented}")
    print(f"Prediksi Label (Original): {predicted_label_original}")
    print()

Age Range 1
image 20
Prediksi Label (Augmented): 1
Prediksi Label (Original): 1

image 21
Prediksi Label (Augmented): 2
Prediksi Label (Original): 1

image 22
Prediksi Label (Augmented): 3
Prediksi Label (Original): 1

image 23
Prediksi Label (Augmented): 3
Prediksi Label (Original): 1

image 24
Prediksi Label (Augmented): 4
Prediksi Label (Original): 1

image 25
Prediksi Label (Augmented): 3
Prediksi Label (Original): 1

image 26
Prediksi Label (Augmented): 1
Prediksi Label (Original): 1

image 27
Prediksi Label (Augmented): 3
Prediksi Label (Original): 1

image 28
Prediksi Label (Augmented): 3
Prediksi Label (Original): 1



In [ ]:
print("Age Range 2")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/2/{i}.jpg'  # Update this path as needed
    predicted_label_augmented = classify_image(image_path, model_rf, augment=True)
    print(f"image {i}")
    print(f"Prediksi Label (Augmented): {predicted_label_augmented}")
    print(f"Prediksi Label (Original): {predicted_label_original}")
    print()

Age Range 2
image 20
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 21
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 22
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 23
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 24
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 25
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 26
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 27
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 28
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1



In [ ]:
print("Age Range 3")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/3/{i}.jpg'  # Update this path as needed
    predicted_label_augmented = classify_image(image_path, model_rf, augment=True)
    print(f"image {i}")
    print(f"Prediksi Label (Augmented): {predicted_label_augmented}")
    print(f"Prediksi Label (Original): {predicted_label_original}")
    print()

Age Range 3
image 20
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 21
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 22
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 23
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 24
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 25
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 26
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 27
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 28
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1



In [ ]:
print("Age Range 4")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/4/{i}.jpg'  # Update this path as needed
    predicted_label_augmented = classify_image(image_path, model_rf, augment=True)
    print(f"image {i}")
    print(f"Prediksi Label (Augmented): {predicted_label_augmented}")
    print(f"Prediksi Label (Original): {predicted_label_original}")
    print()

Age Range 4
image 20
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 21
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 22
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 23
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 24
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 25
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 26
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 27
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 28
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1



In [ ]:
print("Age Range 5")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/5/{i}.jpg'  # Update this path as needed
    predicted_label_augmented = classify_image(image_path, model_rf, augment=True)
    print(f"image {i}")
    print(f"Prediksi Label (Augmented): {predicted_label_augmented}")
    print(f"Prediksi Label (Original): {predicted_label_original}")
    print()

Age Range 5
image 20
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 21
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 22
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 23
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 24
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 25
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 26
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 27
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

image 28
Prediksi Label (Augmented): 5
Prediksi Label (Original): 1

